# Proyecto DataFrames

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [0]:
df = spark.sql("SELECT * FROM pyspark.default.walmart_stock")

In [0]:
df.show()

#### What are the column names?

In [0]:
df.columns

#### What does the Schema look like?

In [0]:
df.printSchema()

#### Print out the first 5 rows.

In [0]:
df.show(5)

In [0]:
df.head(5)

#### Use describe() to learn about the DataFrame.

In [0]:
df.describe().show()

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [0]:
df.printSchema()

In [0]:
from pyspark.sql import functions as F

In [0]:
df.describe().show()

In [0]:
result = df.describe()
result.select(result['summary'],
              F.format_number(result['Open'].try_cast('float'),2).alias('Open'),
              F.format_number(result['High'].try_cast('float'),2).alias('High'),
              F.format_number(result['Low'].try_cast('float'),2).alias('Low'),
              F.format_number(result['Close'].try_cast('float'),2).alias('Close'),
              result['Volume'].try_cast('int').alias('Volume')
             ).show()

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [0]:
df1 = df.withColumn("HV Ratio", df["High"] / df["Volume"])
# df1["HV Ratio"]
df1.select("HV Ratio").show()

In [0]:
display(df1)

#### What day had the Peak High in Price?

In [0]:
df.orderBy(df["High"].desc()).head(1)[0][0]

#### What is the mean of the Close column?

In [0]:
df.agg(F.avg("Close")).show()

#### What is the max and min of the Volume column?

In [0]:
df.agg(F.min("Volume"), F.max("Volume")).show()

#### How many days was the Close lower than 60 dollars?

In [0]:
df.filter("Close<60").count()

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [0]:
numberOfDays = df.filter(df["High"]>80).count()
percentage = (numberOfDays/df.count())*100
percentage

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [0]:
df.select(F.corr("High", "Volume")).show()

#### What is the max High per year?

In [0]:
years = df.withColumn("Year", F.year(df["Date"]))
maximum = years.groupBy("Year").max()
maximum.select("Year", "max(High)").show()

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [0]:
months = df.withColumn("Month", F.month(df["Date"]))
average = months.groupBy("Month").avg()
average.select("Month","avg(Close)").orderBy("Month").show()